In [1]:
using JSON
using JuMP

using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_journal

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in transpose(::Ptr{PyCall.PyObject_struct}) at ./deprecated.jl:770
 in transpose_f!(::Base.#transpose, ::Array{Ptr{PyCall.PyObject_struct},2}, ::Array{Ptr{PyCall.PyObject_struct},2}) at ./arraymath.jl:369
 in transpose(::Array{Ptr{PyCall.PyObject_struct},2}) at ./arraymath.jl:407
 in copy(::PyCall.PyArray{Ptr{PyCall.PyObject_struct},2}) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:337
 in convert(::Type{Array{Ptr{PyCall.PyObject_struct},N}}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:453
 in convert(::Type{Array{PyCall.PyObject,N}}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:484
 in convert(::Type{PyCall.PyAny}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/conversions.jl:806
 in #55 at ./<missing>:0 [inlined]
 in next at ./generator.jl:26 [inlined]
 in collect_to!(::Array{Expr,1}, ::Base.Generator{Array{Tuple{AbstractString,PyCall.PyObject},1},PyCall.##55#59}, ::Int64, ::Int64) at

In [2]:
include("../Julia_files/load_network_uni_class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
numLinks = 258;  # number of links
numNodes = 74;  # number of nodes

74

In [4]:
# #load OD pair-route incidence
# odPairRoute = readstring("od_pair_route_incidence_MA_journal.json");
# odPairRoute = JSON.parse(odPairRoute);

# #load link-route incidence
# linkRoute = readstring("link_route_incidence_MA_journal.json");
# linkRoute = JSON.parse(linkRoute);
# #load OD pair labels
# odPairLabel = readstring("od_pair_label_dict_MA_refined_journal.json");
# odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = Dict()
idx = 1
for origi = 1:numNodes
    for desti = 1:numNodes
        if origi != desti
            odPairLabel_["$(idx)"] = [origi,desti]
            idx += 1
        end
    end
end

In [5]:
odPairLabel_

Dict{Any,Any} with 5402 entries:
  "4304" => [59,71]
  "3935" => [54,67]
  "2243" => [31,54]
  "1881" => [26,57]
  "4209" => [58,48]
  "1907" => [27,9]
  "2923" => [41,3]
  "599"  => [9,16]
  "2491" => [35,9]
  "228"  => [4,10]
  "2590" => [36,35]
  "3697" => [51,47]
  "5031" => [69,67]
  "2579" => [36,24]
  "1880" => [26,56]
  "2562" => [36,7]
  "3215" => [45,3]
  "3991" => [55,49]
  "4652" => [64,53]
  "4088" => [56,74]
  "3841" => [53,45]
  "1106" => [16,11]
  "2366" => [33,30]
  "3196" => [44,58]
  "1783" => [25,32]
  ⋮      => ⋮

In [6]:
# #load link labels
# linkLabel = readstring("link_label_dict_MA_journal.json");
# linkLabel = JSON.parse(linkLabel);

# linkLabel_ = readstring("link_label_dict_MA__journal.json");
# linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readstring("../temp_files/node_link_incidence_MA_journal.json");
nodeLink = JSON.parse(nodeLink);

ta_data = load_ta_network("MA_journal_Apr_PM");

In [7]:
nodeLink

Dict{String,Any} with 19092 entries:
  "2-21"   => 0.0
  "51-171" => 0.0
  "22-4"   => 0.0
  "10-128" => 0.0
  "30-139" => 0.0
  "68-33"  => 0.0
  "33-67"  => 0.0
  "24-251" => 0.0
  "45-61"  => 0.0
  "3-257"  => 0.0
  "9-60"   => 0.0
  "22-86"  => 0.0
  "2-215"  => 0.0
  "11-52"  => 0.0
  "49-247" => 0.0
  "40-215" => 0.0
  "66-187" => 0.0
  "28-136" => 0.0
  "30-131" => 0.0
  "63-35"  => 0.0
  "49-106" => 0.0
  "35-1"   => 0.0
  "47-113" => 0.0
  "71-10"  => 0.0
  "19-141" => 0.0
  ⋮        => ⋮

In [8]:
deg = 8
c = 0.5
lam = 10000.0

coeffs_dict_Apr_PM_ = readstring("../temp_files/coeffs_dict_Apr_PM.json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

println(fcoeffs)

capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

Any[1.0,-0.00303133,0.0577207,-0.195677,0.620789,-0.905919,0.935921,-0.469131,0.108528]


In [9]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + 
        free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + 
        free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + 
        free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + 
        free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + 
        free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 + 
        free_flow_time[a] * fcoeffs[8] * linkFlowVec[a]^8 / capacity[a]^7 + 
        free_flow_time[a] * fcoeffs[9] * linkFlowVec[a]^9 / capacity[a]^8 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [1]:
include("../Julia_files/tap_MSA.jl");

LoadError: LoadError: LoadError: UndefVarError: start_node not defined
while loading /home/jzh/Research/Traffic/InverseVIsTraffic/Julia_files/tap_MSA.jl, in expression starting on line 33
while loading In[1], in expression starting on line 1

In [ ]:
demandsDict = readstring("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM_journal/results/demandsDictFixed$(day)_journal.json");
demandsDict = JSON.parse(demandsDict);

tapFlowDicDict = {}
tapFlowVecDict = {}
tapFlowDicDict[2], tapFlowVecDict[2] = tapMSA(demandsDict, fcoeffs);

In [10]:
function POA_MA_Apr_PM_journal_alt(day)

    demandsDict = readstring("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM_journal/results/demandsDictFixed$(day)_journal.json");
    demandsDict = JSON.parse(demandsDict);

    # demands = demandsDict["$(length(demandsDict)-1)"]

    demands = demandsDict
    
    demands_ = Dict()

    for key in keys(demands)
        key_ = (parse(Int, split(split(key, ',')[1], '(')[2]),parse(Int, split(split(key, ',')[2], ')')[1]))
        demands_[key_] = demands[key]
    end

#     odPairLabel_;

    demandsVec = zeros(5402)

    for i = 1:length(demandsVec)
        demandsVec[i] = demands["($(odPairLabel_["$i"][1]),$(odPairLabel_["$i"][2]))"]
    end

#     demandsVec

#     for key=keys(odPairRoute)
#         if contains(key, "5402-")
#             println(key)
#         end
#     end

#     linkRoute;



    # fcoeffs = [1, 0, 0, 0, .15]
    
    # m = Model(solver=GurobiSolver(OutputFlag=false))
    m = Model()

    @variable(m, linkFlow[1:numLinks])

    @variable(m, odLinkFlow[keys(demands_), 1:numLinks])
    for (s,t) = keys(demands_)
        for k = 1:numLinks
            @constraint(m, odLinkFlow[(s,t), k] >= 0)
        end
    end

    for k = 1:numLinks
        linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands_)])
        @constraint(m, linkFlowPartial == linkFlow[k])
    end

    for l = 1:numNodes
        for (s,t) = keys(demands_)
            if s != t
                odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
                if (l == s)
                    @constraint(m, odLinkFlowPartial + demands_[(s,t)] == 0)
                elseif (l == t)
                    @constraint(m, odLinkFlowPartial - demands_[(s,t)] == 0)
                else
                    @constraint(m, odLinkFlowPartial == 0)
                end
            end
        end
    end

    @NLexpression(m, f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
        free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
        free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
        free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
        free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
        free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
        free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6 +
        free_flow_time[a] * fcoeffs[8] * linkFlow[a]^8 / capacity[a]^7 +
        free_flow_time[a] * fcoeffs[9] * linkFlow[a]^9 / capacity[a]^8, a = 1:numLinks})

    @NLobjective(m, Min, f)

    solve(m)

    println(getvalue(linkFlow))

    getobjectivevalue(m)

    flows = Dict()

    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        flows[key] = getvalue(linkFlow)[i]
    end

#     flows

    # getvalue(linkFlow)

    # getobjectivevalue(m)

    flow_user = GLS_Apr_weekday_PM_journal.x_
    
    println(socialObj(flow_user[:, day])/getobjectivevalue(m))
    
    return socialObj(flow_user[:, day])/getobjectivevalue(m)
end

POA_MA_Apr_PM_journal_alt (generic function with 1 method)

In [11]:
# week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
week_day_Apr_list = [2]

1-element Array{Int64,1}:
 2

In [12]:
poaDictAprPM_journal = Dict()

Dict{Any,Any} with 0 entries

In [13]:
# for day in week_day_Apr_list
#     poaDictAprPM[day] = POA_MA_Apr_PM_journal(day)
# end

In [14]:
for day in week_day_Apr_list
    poaDictAprPM_journal[day] = POA_MA_Apr_PM_journal_alt(day)
end

LoadError: LoadError: InterruptException:
while loading In[14], in expression starting on line 1

In [15]:
outfile = open("./results/poaDictAprPM_journal.json", "w")

JSON.print(outfile, poaDictAprPM_journal)

close(outfile)

In [16]:
poaDictAprPM_journal

Dict{Any,Any} with 0 entries